Week 4

In [7]:
import pandas as pd
import altair as alt

# Updated data with Week 4 rankings
data = {
    "Team": [
        "Two Curls One Kupp",
        "Ladies Love Cool Jalen",
        "Njigbas in Paris",
        "Ass Etienne Szn",
        "Raheem's Rizzlers",
        "Team10Chrysler",
        "Big Dak Energy",
        "Butker's Housewife Brigade",
        "Mike Vick's Underdogs",
        "The Dicker McDickersons",
        "Two Curls One Kupp",
        "Raheem's Rizzlers",
        "Ass Etienne Szn",
        "Njigbas in Paris",
        "Team10Chrysler",
        "Ladies Love Cool Jalen",
        "Big Dak Energy",
        "Butker's Housewife Brigade",
        "Mike Vick's Underdogs",
        "The Dicker McDickersons",
        "Ass Etienne Szn",
        "Njigbas in Paris",
        "Ladies Love Cool Jalen",
        "Butker's Housewife Brigade",
        "Team10Chrysler",
        "Mike Vick's Underdogs",
        "Raheem's Rizzlers",
        "Big Dak Energy",
        "The Dicker McDickersons",
        "Two Curls One Kupp",
        "Njigbas in Paris",
        "Butker's Housewife Brigade",
        "Ass Etienne Szn",
        "Ladies Love Cool Jalen",
        "Mike Vick's Underdogs",
        "Raheem's Rizzlers",
        "Team10Chrysler",
        "Big Dak Energy",
        "The Dicker McDickersons",
        "Two Curls One Kupp",
    ],
    "Week": [
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        1,
        2,
        2,
        2,
        2,
        2,
        2,
        2,
        2,
        2,
        2,
        3,
        3,
        3,
        3,
        3,
        3,
        3,
        3,
        3,
        3,
        4,
        4,
        4,
        4,
        4,
        4,
        4,
        4,
        4,
        4,
    ],
    "Ranking": [
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
    ],
}

df = pd.DataFrame(data)

# Calculate the change in rankings for each team
df["Ranking_Change"] = df.groupby("Team")["Ranking"].diff().fillna(0).astype(int)

# Reverse the signs: a lower ranking number (better) should be positive
df["Ranking_Change_Reversed"] = -df["Ranking_Change"]

# Format the ranking change to show + for rise and - for fall
df["Ranking_Change_Formatted"] = df["Ranking_Change_Reversed"].apply(
    lambda x: f"+{x}" if x > 0 else (f"{x}" if x < 0 else "0")
)

# Filter the most recent week's data (week 2 in this case)
latest_week = df["Week"].max()
df_latest = df[df["Week"] == latest_week]

# Create a line chart
lines = (
    alt.Chart(df)
    .mark_line(point=True)
    .encode(
        x=alt.X(
            "Week:O", title="Week", axis=alt.Axis(labelAngle=0)
        ),  # Rotate labels to vertical
        y=alt.Y(
            "Ranking:Q", scale=alt.Scale(domain=[1, 10], reverse=True), title="Ranking"
        ),  # Reverse scale for rankings
        color="Team:N",
        tooltip=["Team", "Week", "Ranking", "Ranking_Change_Formatted"],
    )
    .properties(width=600, height=400)
)

# Add text labels to the most recent week's data points
text = (
    alt.Chart(df_latest)
    .mark_text(align="left", dx=5, dy=-5)
    .encode(
        x=alt.X("Week:O"),
        y=alt.Y("Ranking:Q"),
        text="Team:N",
        color="Team:N",  # Keep team colors consistent
    )
)

# Combine the line chart with the text labels
chart = (
    (lines + text)
    .configure_legend(disable=True)  # Disable the legend at the top level
    .properties(title="Power Rankings Tracker")
)

# Save the chart
chart.save("../img/rankings_tracker.html")

In [8]:
chart

alt.LayerChart(...)